# DubsTech Data Hackathon 2023


## Track: Retail Analysis

Question: How to gain back $40k

ML: Can do what-if forecast
DA: Basket Analysis : Which items should be paired togehter?


Suru: When people purchase oil, the profit increases too

Importing Libraries

In [113]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import seaborn.objects as so
from apyori import apriori
from mlxtend.frequent_patterns import apriori as apriori_mlx

Importing the Dataset

Source: https://share-docs.clickup.com/26455927/d/h/t7bvq-348/40c04786de66f7b/t7bvq-1728

In [6]:
df = pd.read_csv("Data/sales_data_2017_2018_for_tableau_with_new_date_columns.csv")
print("Length of Dataset: ", len(df))
df.head()

Length of Dataset:  372757


,receipt_id,date,hour,quarter,year,month_number,month_name,day_of_week_name,week_number,is_weekday,...,main_category,sub_category,quantity,payment_type,unit_buying_price,unit_selling_price,unit_price_margin,total_buying_price,total_selling_price,total_profit
0,14b5b35b-4155-45c5-9fa1-58e81d508a25,4/2/2018 2:16:32 PM,14,2,2018,4,April,Monday,14,1,...,Fresh Produce,Tomatoes,0.132,cash,0.0,4.99,4.99,0.0,0.65868,0.65868
1,45755456-0890-450a-af1b-b10b0c197af4,1/25/2018 11:54:20 AM,11,1,2018,1,January,Thursday,4,1,...,Fresh Produce,Tomatoes,0.134,cash,0.0,4.99,4.99,0.0,0.66866,0.66866
2,48910672-6e70-4c1a-8efc-e348c45d519c,4/13/2018 5:40:15 PM,17,2,2018,4,April,Friday,15,1,...,Fresh Produce,Tomatoes,0.136,cash,0.0,4.99,4.99,0.0,0.67864,0.67864
3,dd2882f2-4211-4828-bccb-b53821d29559,1/11/2018 1:44:42 PM,13,1,2018,1,January,Thursday,2,1,...,Fresh Produce,Tomatoes,0.138,cash,0.0,4.99,4.99,0.0,0.68862,0.68862
4,142d4d58-c63b-4fff-80c0-da43e87a2070,1/18/2018 2:28:24 PM,14,1,2018,1,January,Thursday,3,1,...,Fresh Produce,Tomatoes,0.144,cash,0.0,4.99,4.99,0.0,0.71856,0.71856


### We first perform some Descriptive Analysis to understand the dataset

In [7]:
df.isna().sum()

receipt_id                 0
date                       0
hour                       0
quarter                    0
year                       0
month_number               0
month_name                 0
day_of_week_name           0
week_number                0
is_weekday                 0
is_weekend                 0
item_code                  0
item_name                  0
main_category              0
sub_category               0
quantity                   0
payment_type               0
unit_buying_price      86409
unit_selling_price         0
unit_price_margin          0
total_buying_price     86409
total_selling_price        0
total_profit               0
dtype: int64

###### It is observed that 2 key columns have missing value: unit_buying_price and total_buying_price which is based on the earlier column. Since 23% of the data is missing, we will perform some Data Imputation steps before proceeding.

## Data Cleaning


1.1 During our analysis, it is also observed that there are 2 columns- "Beverages" and "Beverage" that we can unify as one

1.2 In the sub_category column, many observations were misspelled. We replace the incorrect values

In [137]:
# 1.1 

df['main_category'].replace('Beverages','Beverage', regex=True, inplace = True)


# 2.1

df['sub_category'].replace('Biscuits','Biscuit', regex=True, inplace = True)
df['sub_category'].replace('Cocunut Products','Coconut Products', regex=True, inplace = True)
df['sub_category'].replace('Condiments','Condiment', regex=True, inplace = True)
df['sub_category'].replace('Packaed','Packaged', regex=True, inplace = True)
df['sub_category'].replace('Sauces','Sauce', regex=True, inplace = True)
df['sub_category'].replace('Spides','Spices', regex=True, inplace = True)
df['sub_category'].replace('Sweets','Sweet', regex=True, inplace = True)
df['sub_category'].replace('spices','Spices', regex=True, inplace = True)


# Convert all Item_names to Sentence Case

df['item_name'] = df['item_name'].str.title()

# Implementing Frequent Itemset Mining using Apriori Algorithm

In [30]:
subset = df[['receipt_id','item_name']]

In [31]:
subset.head()

,receipt_id,item_name
0,14b5b35b-4155-45c5-9fa1-58e81d508a25,Tomatoes Roma
1,45755456-0890-450a-af1b-b10b0c197af4,Tomatoes Roma
2,48910672-6e70-4c1a-8efc-e348c45d519c,Tomatoes Roma
3,dd2882f2-4211-4828-bccb-b53821d29559,Tomatoes Roma
4,142d4d58-c63b-4fff-80c0-da43e87a2070,Tomatoes Roma


In [34]:
df[df["receipt_id"]=='14b5b35b-4155-45c5-9fa1-58e81d508a25']['item_name']

0                    Tomatoes Roma
18713             Banana Cavendish
24341          Watermelon seedless
43644     Pure coconut water 520ml
111333            Grapefruit Marsh
138662                   Onion red
149118             Avocado Shepard
184846                Orange navel
191613           Cucumber Lebanese
Name: item_name, dtype: object

In [42]:
from collections import Counter

subdict = Counter(subset)

In [62]:
d = {}
for i in range(len(subset)):
    
    receipt = subset.iloc[i,0]
    item = subset.iloc[i,1]
    
    if receipt not in d:
        d[receipt] = [item]
        
    else:
        d[receipt].append(item)

In [65]:
itemset = []

for val in d.values():
    itemset.append(val)

itemset 

[['Tomatoes Roma',
  'Banana Cavendish',
  'Watermelon seedless',
  'Pure coconut water 520ml',
  'Grapefruit Marsh',
  'Onion red',
  'Avocado Shepard',
  'Orange navel',
  'Cucumber Lebanese'],
 ['Tomatoes Roma',
  'Garlic loose',
  'Olive Oil Rusk 350g',
  'Onion spring',
  'Potato brushed',
  'Vanila Twists 250g',
  'Onion red'],
 ['Tomatoes Roma', 'Eggs 700g'],
 ['Tomatoes Roma',
  'Banana Cavendish',
  'Nectarine White',
  'Potato sweet Gold',
  'Lettuce Iceberg',
  'Cucumber Lebanese'],
 ['Tomatoes Roma',
  'Spinach Lettuce Mix',
  'Celery Pieces',
  'Mushroom Cup',
  'Red Radish bunch',
  'Carrots',
  'Alfa Alfa',
  'Cucumber Lebanese',
  'Capsicum green'],
 ['Tomatoes Roma',
  'Banana Cavendish',
  'Kensington Pride Mangoes large',
  'Berries Strawberry (medium)'],
 ['Tomatoes Roma',
  'potatoes Red 5kg',
  'Plums Angelino',
  'Apricots',
  'Zucchini green',
  'Peaches Yellow',
  'Peach clingstone'],
 ['Tomatoes Roma', 'Onion spring'],
 ['Tomatoes Roma',
  'Carrots',
  'Sweet 

## Apriori - Method 1

In [104]:
#applying apriori algorithm
association_rules = apriori(itemset, min_support=0.0045, min_confidence=0.1, min_lift=3, min_length=2)
association_results = list(association_rules)

In [105]:
for i in range(0, len(association_results)):
    print(association_results[i][0])

frozenset({'Pear Packham', 'Apples Pink Lady'})
frozenset({'Broccoli', 'Beans green'})
frozenset({'Beans green', 'Carrots'})
frozenset({'Beans green', 'Carrots 1kg'})
frozenset({'Beans green', 'Zucchini green'})
frozenset({'Broccoli', 'Carrots'})
frozenset({'Broccoli', 'Carrots 1kg'})
frozenset({'Broccoli', 'Cauliflower'})
frozenset({'Broccoli', 'Cauliflower half'})
frozenset({'Broccoli', 'Mushroom Cup'})
frozenset({'Broccoli', 'Zucchini green'})
frozenset({'Capsicum green', 'Capsicum red'})
frozenset({'Mushroom Cup', 'Capsicum red'})
frozenset({'Zucchini green', 'Capsicum red'})
frozenset({'Onion brown', 'Carrots'})
frozenset({'Zucchini green', 'Carrots'})
frozenset({'Zucchini green', 'Carrots 1kg'})
frozenset({'Lettuce Iceberg', 'Cucumber continental'})
frozenset({'Garlic loose', 'Ginger'})
frozenset({'Mushroom Cup', 'Zucchini green'})
frozenset({'Potato washed', 'Onion brown'})
frozenset({'Broccoli', 'Capsicum red', 'Banana Cavendish'})
frozenset({'Broccoli', 'Field Tomatoes', 'Bana

In [106]:
for item in association_results:
    # first index of the inner list
    # Contains base item and add item
    pair = item[0]
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1],end=" ")
    
    try:
        print(" -> " + items[2],end=" ")
        print(" -> " + items[3])
    except:
        print("")
        
    # second index of the inner list
    print("Support: " + str(item[1]))
    # third index of the list located at 0th position
    # of the third index of the inner list
    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("-----------------------------------------------------")

Rule: Pear Packham -> Apples Pink Lady 
Support: 0.005584335585244429
Confidence: 0.12055809897536517
Lift: 3.1541629768120174
-----------------------------------------------------
Rule: Broccoli -> Beans green 
Support: 0.008341159481757501
Confidence: 0.20531941337310466
Lift: 3.8801842649042815
-----------------------------------------------------
Rule: Beans green -> Carrots 
Support: 0.0061902309471154335
Confidence: 0.15237385036042755
Lift: 3.8482849476261314
-----------------------------------------------------
Rule: Beans green -> Carrots 1kg 
Support: 0.00461490300625082
Confidence: 0.11359681829480486
Lift: 3.254962999212859
-----------------------------------------------------
Rule: Beans green -> Zucchini green 
Support: 0.006008462338554132
Confidence: 0.14789957742977877
Lift: 3.1908608832546195
-----------------------------------------------------
Rule: Broccoli -> Carrots 
Support: 0.007573692023387561
Confidence: 0.1431297709923664
Lift: 3.6148206661721676
-----------